In [1]:
from binance import Client
import requests
from dotenv import load_dotenv
import os
import ta
import pandas as pd
from time import sleep
from binance.exceptions import BinanceAPIException
import time
from datetime import datetime

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_SECRET_KEY")

In [4]:
client = Client(api_key, api_secret)

In [5]:
pd.DataFrame(client.get_historical_klines('ETHBUSD', '1h', '10h ago UTC'))

,0,1,2,3,4,5,6,7,8,9,10,11
0,1639339200000,4142.15000000,4175.15000000,4140.03000000,4149.83000000,3026.78910000,1639342799999,12588970.86040400,9831,1795.25010000,7466035.58307900,0
1,1639342800000,4149.84000000,4157.32000000,4116.77000000,4124.96000000,2784.20420000,1639346399999,11522009.87711200,9890,1342.08600000,5553441.17471500,0
2,1639346400000,4124.97000000,4156.99000000,4117.74000000,4156.64000000,2155.72510000,1639349999999,8916063.16988400,8296,1154.48870000,4774937.34086800,0
3,1639350000000,4156.58000000,4170.00000000,4126.51000000,4132.00000000,3351.27230000,1639353599999,13900224.04814300,11304,1793.49220000,7439917.79424500,0
4,1639353600000,4131.99000000,4144.53000000,4057.19000000,4069.97000000,7214.69670000,1639357199999,29551745.84446500,21022,2522.12950000,10337843.80570400,0
5,1639357200000,4070.36000000,4070.36000000,3969.14000000,3971.30000000,16353.22580000,1639360799999,65620763.88686700,42176,6289.68000000,25248569.08256600,0
6,1639360800000,3971.55000000,3998.93000000,3957.37000000,3983.41000000,5477.79510000,1639364399999,21797751.26201100,17013,2681.06350000,10669634.72701500,0
7,1639364400000,3983.98000000,4010.89000000,3974.31000000,3998.40000000,3570.86560000,1639367999999,14265375.61158800,11131,1679.71960000,6708930.51412800,0
8,1639368000000,3998.71000000,4024.00000000,3991.05000000,4019.77000000,2873.47770000,1639371599999,11527447.49553800,9809,1509.39330000,6056497.04578900,0
9,1639371600000,4019.63000000,4021.82000000,4012.03000000,4012.97000000,289.37140000,1639375199999,1162057.62287200,1054,135.65700000,544774.33359000,0


In [6]:
def getdata(symbol):
    try:
        frame = pd.DataFrame(client.get_historical_klines(symbol, '1h', '10h UTC'))
    except BinanceAPIException as e:
        print(e)
        sleep(60)
        frame = pd.DataFrame(client.get_historical_klines(symbol, '1h', '10h UTC'))
    frame = frame.iloc[:, :6]
    frame.columns = ['Time','Open', 'High','Low','Close','Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

                               

In [7]:
getdata('ETHBUSD')

,Open,High,Low,Close,Volume
Time,,,,,
2021-12-12 20:00:00,4142.15,4175.15,4140.03,4149.83,3026.7891
2021-12-12 21:00:00,4149.84,4157.32,4116.77,4124.96,2784.2042
2021-12-12 22:00:00,4124.97,4156.99,4117.74,4156.64,2155.7251
2021-12-12 23:00:00,4156.58,4170.00,4126.51,4132.00,3351.2723
2021-12-13 00:00:00,4131.99,4144.53,4057.19,4069.97,7214.6967
2021-12-13 01:00:00,4070.36,4070.36,3969.14,3971.30,16353.2258
2021-12-13 02:00:00,3971.55,3998.93,3957.37,3983.41,5477.7951
2021-12-13 03:00:00,3983.98,4010.89,3974.31,3998.40,3570.8656
2021-12-13 04:00:00,3998.71,4024.00,3991.05,4019.77,2873.4777


In [8]:
def tradingstrat(symbol, open_position = False):
    while True:
        df = getdata(symbol)      
        if not open_position:
            if ta.trend.macd_diff(df.Close).iloc[-1] > 0 \
            and ta.trend.macd_diff(df.Close).iloc[-2] < 0:
                balance = client.get_asset_balance(asset='BUSD')
                balance= balance['free']
                balance = float(balance)
                trade_size = (balance * .30)/(random.Close).iloc[-1]
                
                order = client.create_order(symbol=symbol,
                                               side='BUY',
                                               type='MARKET',
                                               quantity=trade_size)
                print(order)
                open_position = True
                buyprice = float(order['fills'][0]['price'])
                break 
                
        if open_position:
            while True:
                df = getdata(symbol)
                
                if ta.trend.macd_diff(df.Close).iloc[-1] < 0 \
                and ta.trend.macd_diff(df.Close).iloc[-2] > 0 | df.Close[-1] <= buyprice * 0.90 or df.Close >= 2 * buyprice:
                    order = client.create_order(symbol=symbol,
                                               side='BUY',
                                               type='MARKET',
                                               quantity=trade_size) 
                    print(order)
                    sellprice = float(order['fills'][0]['price'])
                    print(f'profit = {(sellprice - buyprice)/buyprice}')
                    open_position = False
                    break

In [9]:
while True:
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print(dt_string)
    sleep(60)
    tradingstrat('ETHBUSD')
    

13/12/2021 16:07:17


KeyboardInterrupt: 

In [ ]:
# APIError(code=-1001): Internal error; unable to process your request. Please try again.